In [2]:
import pickle

In [3]:
dir = './features/'

In [4]:
city_info = ['date_label','geo_label_city']

strata_info = ['encoded_strata_race_label', 'encoded_strata_sex_label',
       'encoded_geo_strata_region', 'encoded_geo_strata_poverty',
       'encoded_geo_strata_Population', 'encoded_geo_strata_PopDensity',
       'encoded_geo_strata_Segregation', 'date_label']

features_list = []

target_cand = ['Cancer | All Cancer Deaths',
 'Cancer | Colorectal Cancer Deaths', 
 'Cancer | Lung Cancer Deaths', 
 'Cardiovascular Disease | Cardiovascular Disease Deaths',
 'Cardiovascular Disease | Heart Disease Deaths',
 'Deaths | Deaths from All Causes',
 'Deaths | Gun Deaths (Firearms)',
 'Deaths | Injury Deaths',
 'Deaths | Motor Vehicle Deaths',
 'Deaths | Premature Death',
 'Diabetes and Obesity | Diabetes Deaths',
 'Life Expectancy at Birth | Life Expectancy',
 'Mental Health | Suicide',
 'Substance Use | Drug Overdose Deaths']

target_list = target_cand

# 결측치가 너무 많아/불균형해 제외된 column
avoid_list_orig = ['Substance Use | Teen Smoking',
 'Diabetes and Obesity | Teen Obesity',
 'Substance Use | Teen Alcohol',
 'Physical Activity | Teen Computer Time',
 'Physical Activity | Teen Physical Activity',
 'Substance Use | Teen Marijuana',
 'Physical Activity | Teen TV Time',
 'Dietary Quality | Teen Soda',
 'Mental Health | Electronic Bullying',
 'Mental Health | School Bullying',
 'Mental Health | Teen Mental Distress',
 'Mental Health | Teen Suicidal Ideation',
 'Physical Activity | Teen Physical Inactivity',
 'Respiratory Disease | Teen Asthma',
 'School Violence | Fighting in School',
 'School Violence | Weapons in School',
 'Birth Control | Teen Birth Control',
 'Dietary Quality | Teen Breakfast',
 'Physical Activity | Teen Physical Education',
 'Respiratory Infection | COVID-19 Deaths',
 'Respiratory Infection | COVID-19 Vaccination, Adults',
 'Respiratory Infection | COVID-19 Vaccination, Seniors',
 'Births | Prenatal Care',
 'Crime Incidents | Violent Crime',
 'Lead Poisoning | Child Lead Levels 10+ mcg/dL',
 'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
 'Lead Poisoning | Child Lead Testing']

# 일부 target 따라 너무 비슷한 값(ex) All Cancer를 예측할 때 세분화된 Cancer 사망자)
targetspecific_list = {
    'Cancer | All Cancer Deaths': ['Cancer | Breast Cancer Deaths', 'Cancer | Colorectal Cancer Deaths','Cancer | Lung Cancer Deaths','Cancer | Prostate Cancer Deaths', 'Deaths | Deaths from All Causes'],
    'Cancer | Colorectal Cancer Deaths' : [],
    'Cancer | Lung Cancer Deaths' : [],
    'Cardiovascular Disease | Cardiovascular Disease Deaths': [],
    'Cardiovascular Disease | Heart Disease Deaths': [],
    'Deaths | Deaths from All Causes': ['Deaths | Infant Deaths', 'Deaths | Maternal Deaths', 'Deaths | Police Killings', 'Cancer | Breast Cancer Deaths', 'Cancer | Colorectal Cancer Deaths','Cancer | Lung Cancer Deaths','Cancer | Prostate Cancer Deaths', 'Crime Incidents | Homicides'],
    'Deaths | Gun Deaths (Firearms)': [],
    'Deaths | Injury Deaths': [],
    'Deaths | Motor Vehicle Deaths': [],
    'Deaths | Premature Death': [],
    'Diabetes and Obesity | Diabetes Deaths': [],
    'Life Expectancy at Birth | Life Expectancy': [],
    'Mental Health | Suicide': [],
    'Substance Use | Drug Overdose Deaths': ['Substance Use | Opioid Overdose Deaths']
}

heuristic_list = {
    'Cancer | All Cancer Deaths': [
        'Cancer | Breast Cancer Deaths',
        'Cancer | Prostate Cancer Deaths',
        'Physical Activity | Adult Physical Inactivity',
        'Diabetes and Obesity | Diabetes',
        'Diabetes and Obesity | Teen Obesity',
        'Diabetes and Obesity | Adult Obesity',
        'Dietary Quality | Teen Soda',
        'Substance Use | Adult Binge Drinking',
        'Population | Seniors',
        'Health Insurance | Uninsured, All Ages',
        'Births | Low Birthweight',
        'Births | Prenatal Care',
        'Births | Preterm Births',
        'Births | Teen Births',
        'Income | Poverty in All Ages'
        ],
    'Cancer | Colorectal Cancer Deaths' : [
        'Cancer | Breast Cancer Deaths',
        'Cancer | Prostate Cancer Deaths',
        'Dietary Quality | Teen Breakfast',
        'Dietary Quality | Teen Soda',
        'Diabetes and Obesity | Adult Obesity',
        'Diabetes and Obesity | Teen Obesity',
        'Dietary Quality | Teen Soda',
        'Physical Activity | Teen Physical Inactivity',
        'Population | Seniors',
        'Health Insurance | Uninsured, All Ages',
        'Births | Low Birthweight',
        'Births | Prenatal Care',
        'Births | Preterm Births',
        'Births | Teen Births',
        'Income | Poverty in All Ages'
    ],
    'Cancer | Lung Cancer Deaths' : [
        'Cancer | Breast Cancer Deaths',
        'Cancer | Prostate Cancer Deaths',
        'Substance Use | Adult Smoking',
        'Substance Use | Teen Smoking',
        'Air Pollution | Hazardous Air Quality',
        'Air Pollution | Poor Air Quality',
        'Population | Seniors',
        'Diabetes and Obesity | Diabetes',
        'Health Insurance | Uninsured, All Ages',
        'Births | Low Birthweight',
        'Births | Prenatal Care',
        'Births | Preterm Births',
        'Births | Teen Births',
        'Income | Poverty in All Ages'
    ],
    'Cardiovascular Disease | Cardiovascular Disease Deaths': [
        'Cardiovascular Disease | High Blood Pressure',
        'Diabetes and Obesity | Adult Obesity',
        'Physical Activity | Adult Physical Inactivity',
        'Physical Activity | Teen Physical Activity',
        'Physical Activity | Teen Physical Inactivity',
        'Substance Use | Adult Smoking',
        'Population | Seniors',
        'Births | Low Birthweight',
        'Births | Prenatal Care',
        'Births | Preterm Births',
        'Births | Teen Births',
        'Dietary Quality | Teen Soda',
        'Dietary Quality | Teen Breakfast'
    ],
    'Cardiovascular Disease | Heart Disease Deaths': [
        'Cardiovascular Disease | High Blood Pressure',
        'Diabetes and Obesity | Adult Obesity',
        'Physical Activity | Adult Physical Inactivity',
        'Physical Activity | Teen Physical Activity',
        'Physical Activity | Teen Physical Inactivity',
        'Substance Use | Adult Smoking',
        'Dietary Quality | Teen Soda',
        'Dietary Quality | Teen Breakfast',
        'Population | Seniors',
        'Health Insurance | Uninsured, All Ages',
        'Births | Low Birthweight',
        'Births | Prenatal Care',
        'Births | Preterm Births',
        'Births | Teen Births',
        'Income | Poverty in All Ages'
    ],
    'Deaths | Deaths from All Causes': [],
    'Deaths | Gun Deaths (Firearms)': [
        'Crime Incidents | Homicides',
        'Crime Incidents | Violent Crime',
        'Deaths | Police Killings',
        'Deaths | Racial Disparity in Police Killings',
        'Mental Health | Teen Suicidal Ideation',
        'Mental Health | Adult Mental Distress',
        'Mental Health | School Bullying',
        'Mental Health | Electronic Bullying',
        'Substance Use | Adult Binge Drinking',
        'School Violence | Weapons in School',
        'Education | College Graduates',
        'Education | Preschool Enrollment'
    ],
    'Deaths | Injury Deaths': [
        'Active Transportation | Riding Bike to Work',
        'Active Transportation | Walking to Work',
        'Transportation | Drives Alone to Work',
        'Transportation | Public Transportation Use',
        'Transportation | Lack of Car',
        'Transportation | Longer Driving Commute Time',
        'Population | Seniors',
        'Deaths | Police Killings',
        'Deaths | Racial Disparity in Police Killings',
        'School Violence | Fighting in School',
        'School Violence | Weapons in School',
        'Transportation | Drives Alone to Work',
        'Transportation | Longer Driving Commute Time',
        'Health Insurance | Uninsured, All Ages',
        'Income | Poverty in All Ages'
    ],
    'Deaths | Motor Vehicle Deaths': [
        'Active Transportation | Riding Bike to Work',
        'Active Transportation | Walking to Work',
        'Transportation | Drives Alone to Work',
        'Transportation | Longer Driving Commute Time',
        'Health Insurance | Uninsured, All Ages',
        'Substance Use | Adult Binge Drinking',
        'Transportation | Drives Alone to Work',
        'Transportation | Longer Driving Commute Time'
    ],
    'Deaths | Premature Death': [
        'Cardiovascular Disease | High Blood Pressure',
        'Diabetes and Obesity | Adult Obesity',
        'Physical Activity | Adult Physical Inactivity',
        'Substance Use | Adult Smoking',
        'Mental Health | Adult Mental Distress',
        'Cancer | Breast Cancer Deaths',
        'Cancer | Prostate Cancer Deaths',
        'Deaths | Infant Deaths',
        'Deaths | Maternal Deaths',
        'Heat and Disasters | Climate-related Disasters',
        'Heat and Disasters | Community Social Vulnerability to Climate Disasters',
        'Heat and Disasters | Longer Summers',
        'Heat and Disasters | Number of Heat Waves',
        'Health Insurance | Uninsured, All Ages',
        'Health Insurance | Uninsured, Child',
        'Income | Poverty in All Ages',
        'Lead Poisoning | Child Lead Levels 10+ mcg/dL',
        'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
        'Lead Poisoning | Child Lead Testing',
        'Lead Poisoning | Housing Lead Risk',
        'Respiratory Infection | COVID-19 Deaths',
        'Respiratory Infection | New Tuberculosis Cases',
        'Respiratory Infection | Pneumonia or Influenza Deaths',
        'School Violence | Fighting in School',
        'School Violence | Weapons in School',
        'Sexually Transmitted Disease | HIV-Related Deaths',
        'Sexually Transmitted Disease | HIV/AIDS Prevalence',
        'Transportation | Drives Alone to Work',
        'Transportation | Longer Driving Commute Time'
    ],
    'Diabetes and Obesity | Diabetes Deaths': [
        'Diabetes and Obesity | Diabetes',
        'Diabetes and Obesity | Adult Obesity',
        'Cardiovascular Disease | High Blood Pressure',
        'Physical Activity | Adult Physical Inactivity',
        'Physical Activity | Teen Physical Activity',
        'Physical Activity | Teen Physical Inactivity',
        'Dietary Quality | Teen Soda',
        'Dietary Quality | Teen Breakfast',
        'Population | Seniors'
    ],
    'Life Expectancy at Birth | Life Expectancy': [
        'Cancer | Prostate Cancer Deaths',
        'Cancer | Breast Cancer Deaths',
        'Cardiovascular Disease | High Blood Pressure',
        'Diabetes and Obesity | Adult Obesity',
        'Cardiovascular Disease | High Blood Pressure',
        'Income | Per-capita Household Income',
        'Lead Poisoning | Child Lead Levels 10+ mcg/dL',
        'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
        'Lead Poisoning | Child Lead Testing',
        'Lead Poisoning | Housing Lead Risk',
        'Substance Use | Teen Alcohol',
        'Substance Use | Teen Marijuana',
        'Substance Use | Teen Smoking'
    ],
    'Mental Health | Suicide': [
        'Mental Health | Adult Mental Distress',
        'Mental Health | Teen Suicidal Ideation',
        'Mental Health | Electronic Bullying',
        'Mental Health | School Bullying',
        'Mental Health | Teen Mental Distress',
        'Births | Teen Births',
        'Substance Use | Adult Binge Drinking',
        'Substance Use | Adult Smoking',
        'Crime Incidents | Homicides',
        'Crime Incidents | Violent Crime',
        'Deaths | Infant Deaths',
        'Deaths | Maternal Deaths',
        'Cardiovascular Disease | High Blood Pressure',
        'Cancer | Breast Cancer Deaths',
        'Cancer | Prostate Cancer Deaths',
        'Income | Poverty in All Ages',
        'Housing | Homeless, Children',
        'Housing | Homeless, Total',
        'Housing | Owner Occupied Housing',
        'Housing | Vacant Housing Units',
        'Housing | Vacant Housing and Homelessness',
        'Income Inequality | Household Income Inequality',
        'Income Inequality | Income Inequality',
        'Income | Households with Higher-Incomes',
        'Income | Per-capita Household Income',
        'Income | Poverty and Near Poverty in All Ages',
        'Income | Poverty in All Ages',
        'Income | Poverty in Children',
        'Income-related | Excessive Housing Cost',
        'Income-related | Public Assistance',
        'Income-related | Service Workers',
        'Income-related | Unemployment',
        'School Violence | Fighting in School',
        'Substance Use | Adult Binge Drinking',
        'Substance Use | Opioid Overdose Deaths',
        'Substance Use | Teen Alcohol',
        'Substance Use | Teen Marijuana',
        'Substance Use | Teen Smoking'
    ],
    'Substance Use | Drug Overdose Deaths': [
        'Substance Use | Opioid Overdose Deaths',
        'Substance Use | Teen Marijuana',
        'Substance Use | Teen Alcohol',
        'Mental Health | Teen Suicidal Ideation',
        'Mental Health | Adult Mental Distress',
        'Mental Health | Electronic Bullying',
        'Mental Health | School Bullying',
        'Mental Health | Teen Mental Distress',
        'Health Insurance | Uninsured, All Ages',
        'Cancer | Breast Cancer Deaths',
        'Cancer | Prostate Cancer Deaths',
        'Income | Poverty in All Ages',
        'Housing | Homeless, Children',
        'Housing | Homeless, Total',
        'Housing | Owner Occupied Housing',
        'Housing | Vacant Housing Units',
        'Housing | Vacant Housing and Homelessness',
        'Income Inequality | Household Income Inequality',
        'Income Inequality | Income Inequality',
        'Income | Households with Higher-Incomes',
        'Income | Per-capita Household Income',
        'Income | Poverty and Near Poverty in All Ages',
        'Income | Poverty in All Ages',
        'Income | Poverty in Children',
        'Income-related | Excessive Housing Cost',
        'Income-related | Public Assistance',
        'Income-related | Service Workers',
        'Income-related | Unemployment',
        'Lead Poisoning | Child Lead Levels 10+ mcg/dL',
        'Lead Poisoning | Child Lead Levels 5+ mcg/dL',
        'Lead Poisoning | Child Lead Testing',
        'Lead Poisoning | Housing Lead Risk',
        'Substance Use | Opioid Overdose Deaths',
        'Substance Use | Teen Alcohol',
        'Substance Use | Teen Marijuana',
        'Substance Use | Teen Smoking'
    ]
}

In [8]:
def save_pickle(target, path):
    with open(path,'wb') as file:
        pickle.dump(target,file)

In [9]:
save_pickle(avoid_list_orig,dir+'avoid_list_orig.pickle')
save_pickle(city_info,dir+'city_info.pickle')
save_pickle(strata_info,dir+'strata_info.pickle')
save_pickle(heuristic_list,dir+'heuristic_list.pickle')
save_pickle(target_cand,dir+'target_cand.pickle')
save_pickle(targetspecific_list,dir+'targetspecific_list.pickle')

In [10]:
#load lists
import pickle

dir = './features/'
with open(dir+'avoid_list_orig.pickle','rb') as f:
    avoid_list_orig = pickle.load(f)
with open(dir+'city_info.pickle','rb') as f:
    city_info = pickle.load(f)
with open(dir+'strata_info.pickle','rb') as f:
    strata_info = pickle.load(f)
with open(dir+'heuristic_list.pickle','rb') as f:
    heuristic_list = pickle.load(f)
with open(dir+'target_cand.pickle','rb') as f:
    target_cand = pickle.load(f)
with open(dir+'targetspecific_list.pickle','rb') as f:
    targetspecific_list = pickle.load(f)

target_list = target_cand